In [63]:
######################  Config start

#Ausgaben speichern unter Name
notebook_name = 'NN-Controller_Example'

#Netzwerkarchitektur
num_hidden_neurons = 5
input_num = 13

#Metaparameter Konfiguration
mue_alpha = 0.08
sigma_alpha = 0.04
num_iterations = 8000
make_plot_each = 50
take_test_data_every = 100
update_feat_every = 500
no_feat_imp_bevore = 2300

#Eingabeparameter Aktivieren oder Deaktivieren
delta_Ti_2h_inp = 1
post_Ta_avg_32h_inp = 1
Tw_inp = 1
Te_inp = 1
price_flag = 1
no_future_inp = 1
dif_price_inp = 0

#Übersetzung der Ausgabe in Vorlauftemperatur
num_outputs = 2
dif = 0
change_rate = 0.02

#Heizungsregelungsbereich
max_t_eval = 23.5
min_t_eval = 22.0

#Kosten Konfiguration
price_factor = 0.5


#########################  Config end



import sys
import os
os.chdir("/XXXX/Daniel")

# Pfad zum Ordner RL_Results
rl_results_path = os.path.join(os.getcwd(), "XXXX")

# Überprüfen, ob der Ordner RL_Results existiert, wenn nicht, erstellen
if not os.path.exists(rl_results_path):
    os.makedirs(rl_results_path)

# Pfad für den neuen Ordner innerhalb von RL_Results
new_folder_path = os.path.join(rl_results_path, notebook_name )

# Erstellen des neuen Ordners, wenn er nicht existiert
if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)

new_folder_path = new_folder_path + "/"

#sys.path.append(r'C:\Users\dhindere\Master-Thesis\Temp_Model\RC-Model_Temp\ZSW_DATA\optimization_lite-main')

from datetime import timedelta 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import random
from matplotlib.ticker import MaxNLocator

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold
from statsmodels.graphics.tsaplots import plot_pacf

import RC_Modell.ownUtils
from RC_Modell.models import DarkGreyModel, Ti, TiTe, TiTeTwSInoSIso
from RC_Modell.plot import plot, plot_input_data, plot_with_Th
from RC_Modell.fit import darkgreyfit
from RC_Modell.error import rmse
from RC_Modell.predict import predict_model

from black_box_optimizers.pgpe import RankSuperSymPGPE
from optimization.optimizer import OptimizeableParameterDict as OP



#Funktionen

def make_weights_biases_from_vector(para_vector, structure):
    weights_biases_list = []
    count = 0
    for i, v in enumerate(structure):
        
        # weights
        cur_weights = np.zeros((v, structure[i+1]), 'float32')
        s = cur_weights.size
        cur_weights = para_vector[count:count+s].reshape(cur_weights.shape)
        count += s
        weights_biases_list.append(cur_weights)
        
        # biases
        cur_weights = np.zeros((structure[i+1],), 'float32')
        s = cur_weights.size
        cur_weights = para_vector[count:count+s].reshape(cur_weights.shape)
        count += s
        weights_biases_list.append(cur_weights)
        
        if i == len(structure)-2:
            break
        
    net_params = {"num_layers": len(structure)-2, "activation": "sigmoid", "out_type": "linear"}
    return weights_biases_list, net_params

def get_num_paras_from_structure(structure):
    count = 0
    for i, v in enumerate(structure):
        # weights
        cur_weights = np.zeros((v, structure[i+1]), 'float32')
        s = cur_weights.size
        count += s
        
        # biases
        cur_weights = np.zeros((1, structure[i+1]), 'float32')
        s = cur_weights.size
        count += s
        
        if i == len(structure)-2:
            break
    return count

def forward_path_nnr(inputs, weights_biases, net_params):
    
    function_mapping = {
        "sigmoid": lambda x: 1 / (1 + np.exp(-x)),
        "leakyReLU": lambda x: ((x > 0) * x) + ((x <= 0) * x * 0.01),
        "tanh": lambda x: np.sinh(x) / np.cosh(x),
        "linear": lambda x: x,
    }

    curr_layer = inputs.copy()
    weights = weights_biases[::2]
    biases = weights_biases[1::2]
    for l in range(net_params["num_layers"]):
        curr_layer = np.dot(curr_layer, weights[l]) + biases[l].reshape(-1,biases[l].shape[-1])
        curr_layer = function_mapping[net_params["activation"]](curr_layer)
    output = np.dot(curr_layer, weights[-1]) + biases[-1]
    output = function_mapping[net_params["out_type"]](output)

    return output

def custom_splits(X, num_sets, train_len, val_len, variance):
    
    total_len = train_len + val_len
    max_index = len(X)
    
    # Calculate step size to distribute the sets across the data
    step = (max_index - total_len) // num_sets
    
    splits = []
    
    for i in range(num_sets):
        # Start index for this set
        start_index = i * step
        
        # Randomly vary the training length
        random_variance = np.random.randint(-variance, variance + 1)
        current_train_len = train_len + random_variance
        
        # Ensure we don't exceed the max index
        if start_index + current_train_len + val_len > max_index:
            break
        
        train_indices = list(range(start_index, start_index + current_train_len))
        val_indices = list(range(start_index + current_train_len, start_index + current_train_len + val_len))
        
        splits.append((np.array(train_indices), np.array(val_indices)))
        
    return splits


def select_model_by_position(df, position):
    model_name = df.iloc[position][('train', 'model')]
    return model_name

def extract_random_sequence(input_df, days):
    # Anzahl der Zeilen, die einem Tag entsprechen (angenommen, die Daten sind stündlich)
    rows_per_day = 24
    
    # Maximaler Startindex, damit die Sequenz von X Tagen nicht den Datenrahmen übersteigt
    max_start_idx = len(input_df) - days * rows_per_day
    
    # Zufälligen Startindex wählen
    start_idx = random.randint(0, max_start_idx)
    
    # Ende-Index berechnen
    end_idx = start_idx + days * rows_per_day
    
    # Sequenz extrahieren
    sequence_df = input_df.iloc[start_idx:end_idx, :]
    
    plot_input_data(sequence_df, with_Kh = False, with_Ph = False)
    
    # Extract input features and target variable
    X_test_test = input_df[['Th', 'Ta', 'SIno', 'SIso', 'Ti0', 'Te0', 'Tw0']]
    y_test_test = input_df['Ti']
    
    return X_test_test, y_test_test

def rc_model_predict(sequence_df, model, error_metric, Te0):
    # Extrahiere die notwendigen Anfangsbedingungen und Parameter
    X = sequence_df.drop('Ti', axis=1)
    y = sequence_df['Ti']
    
    ic_params_map = {
       'Ti0': lambda X, y, model_result: y.iloc[0],
       'Th0': lambda X, y, model_result: y.iloc[0],
       'Te0': lambda X, y, model_result: Te0,    # Verwende den berechneten Wert von Te0
    }
    
    # Führe die Vorhersage durch
    prediction_df = predict_model(model, X, y, ic_params_map, error_metric, model.result)
    predictions = prediction_df['model_result'].iloc[0].var['Ti']
    
    return predictions



def find_min_max_values(df):  
    min_max_values = {}
    all_top_limits = []
    all_bot_limits = []
    überschuss = 0.1

    # Berechnung nur für 'Ti', 'Ta', 'Th'
    for col in ['Ti', 'Ta']:
        min_val = df[col].min()
        max_val = df[col].max()

        dif = max_val - min_val
        top_limit = max_val + dif * überschuss
        bot_limit = min_val - dif * überschuss

        min_max_values[col] = {'min': bot_limit, 'max': top_limit}
        all_top_limits.append(top_limit)
        all_bot_limits.append(bot_limit)

    # Finde den höchsten top_limit und den niedrigsten bot_limit unter 'Ti', 'Ta', 'Th'
    max_top_limit = max(all_top_limits)
    min_bot_limit = min(all_bot_limits)

    # Setze diese Werte für 'Te' und 'Tw'
    min_max_values['Te'] = {'min': min_bot_limit, 'max': max_top_limit}
    min_max_values['Tw'] = {'min': min_bot_limit, 'max': max_top_limit}
    min_max_values['Th'] = {'min': -3, 'max': 3}

    # Füge auch 'SIno' und 'SIso' hinzu, aber ohne sie für 'Te' und 'Tw' zu berücksichtigen
    for col in ['SIno', 'SIso']:
        min_val = df[col].min()
        max_val = df[col].max()

        dif = max_val - min_val
        min_max_values[col] = {'min': min_val - dif * überschuss, 'max': max_val + dif * überschuss}

    # Assuming a reasonable range for deltaTi_-2h and day_hours 
    min_max_values['deltaTi'] = {'min': -2, 'max': 2}
    min_max_values['day_hours'] = {'min': 0, 'max': 24}

    # Using the same range for averages as their original values
    min_max_values['Ta_avg'] = min_max_values['Ta']
    min_max_values['SIso_avg'] = min_max_values['SIso']     
    min_max_values['SIno_avg'] = min_max_values['SIno']

    # For price
    max_price = df['price'].max()
    min_price = df['price'].min()
    if abs(max_price) > abs(min_price):
        price_limit = max_price
    else:
        price_limit = min_price
    min_max_values['price'] = {'min': -price_limit, 'max': price_limit}
    
    return min_max_values

min_max_values = find_min_max_values(input_df)



def normalize_data(value, min_value, max_value):
    return (value - min_value) / (max_value - min_value)

def denormalize_data(norm_value, min_value, max_value):
    return norm_value * (max_value - min_value) + min_value

def evaluate_temp(Ti,time_hour):
    if start_working_hour < 6 or end_working_hour > 20: 
        return 0
    abweichung = 0
    if Ti <= max_t_eval and Ti >= min_t_eval:
        return 0
    elif Ti > max_t_eval:
        abweichung = Ti - max_t_eval
    elif Ti < min_t_eval:
        abweichung = min_t_eval - Ti
    return float(-(abweichung ** 2)*1.5)  # Negative quadratische Abweichung als Belohnung


def evaluate_energy(bkt_an,bkt_vorlauftemp_set,Tw,bkt_state_last,price_now):
    if bkt_an == 0:
        return 0
    initial_cost = 0
    if bkt_state_last == 0:
        initial_cost = 0.01
    abweichung = abs(Tw - bkt_vorlauftemp_set)
    energy_cost = float(-abweichung*0.01-initial_cost)
    if price_flag == 1:
        return energy_cost * price_now * price_factor
    else:
        return energy_cost * price_now * price_factor

    
def evaluate_vorlauf_range(bkt_vorlauftemp):
    #abweichung = 0
    #if bkt_vorlauftemp > (heiz_temp_max+5):
    #    abweichung = bkt_vorlauftemp - (heiz_temp_max+5)
    #elif bkt_vorlauftemp < (kühl_temp_min-5):
    #    abweichung = (kühl_temp_min-5) - bkt_vorlauftemp
    #return float(-(abweichung ** 2) * 0.01)
    return 0


def calculate_bkt_vorlauftemo(normed_nn_out,x_seq_copy,t,next_result):
    
    bkt_vorlauftemp_15 = 0
    bkt_vorlauftemp_30 = 0

    # Reale Heiz/Kühlgrenzen einsetzen
    new_temp = 0

    # Regelung mit Differenzen vom jetztwert:
    if dif == 1:
        if num_outputs == 1:
            if bkt_vorlauftemp <= 5 and bkt_vorlauftemp >= -5:
                new_temp = x_seq_copy['Th'].iloc[t]
            bkt_an = 1
        else:
            if normed_nn_out[0][1] < 0:
                bkt_an = 0
            else:
                bkt_an = 1
                    
        if bkt_an == 1:
            if bkt_vorlauftemp > 5:
                new_temp = x_seq_copy['Th'].iloc[t] + (bkt_vorlauftemp-5) * change_rate
            elif bkt_vorlauftemp < -5:
                new_temp = x_seq_copy['Th'].iloc[t] + (bkt_vorlauftemp+5) * change_rate
    
    else: # Regelung auf Festwert:
        if num_outputs == 1:
            bkt_an = 1
        else:
            if normed_nn_out[0][1] < 0:
                bkt_an = 0
            else:
                bkt_an = 1
                    
        if bkt_an == 1:
            if bkt_vorlauftemp <= 5 and bkt_vorlauftemp >= -5:
                bkt_an = 0
            elif bkt_vorlauftemp < -5:
                new_temp = kühl_temp_max + (bkt_vorlauftemp+5) * change_rate
            elif bkt_vorlauftemp > 5:
                new_temp = heiz_temp_min + (bkt_vorlauftemp-5) * change_rate
                
    # Grenzen gelten für alle:    
    if new_temp > heiz_temp_max:
        new_temp = heiz_temp_max
    elif new_temp < kühl_temp_min:
        new_temp = kühl_temp_min
    elif new_temp > kühl_temp_max and new_temp < heiz_temp_min:
        bkt_an = 0
                    
    bkt_vorlauftemp_15 = new_temp
    bkt_vorlauftemp_30 = new_temp

    # Falls aus: 
    if bkt_an == 0:
        min_in_sec_15min = 15*60
        min_in_sec_30min = 30*60
        bkt_vorlauftemp_15 = use_pt1_function(min_in_sec_15min, beharrungswert=next_result.var['Tw'][-1], start_temp=x_seq_copy['Th'].iloc[t])
        bkt_vorlauftemp_30 = use_pt1_function(min_in_sec_30min, beharrungswert=next_result.var['Tw'][-1], start_temp=x_seq_copy['Th'].iloc[t])

    return bkt_vorlauftemp_15, bkt_vorlauftemp_30


def save_feat_csv(feat_results):
    
    value_names = ["Ti", "Tw", "Te", "delta Ti 2h", "pre Ta avg 12h", "post Ta avg 24h",
                   "pre SIso avg 4h", "post SIso avg 8h", "post Ta avg 32h", "pre Th avg 0.5h",
                   "hour with minutes", "delta Tset", "original"]

    if delta_Ti_2h_inp == 0 and post_Ta_avg_32h_inp == 0 and Tw_inp == 1 and Te_inp == 1:
    
        value_names = ["Ti", "Tw", "Te", "pre Ta avg 12h", "post Ta avg 24h",
                       "pre SIso avg 4h", "post SIso avg 8h",  "pre Th avg 0.5h",
                       "hour with minutes", "delta Tset", "original"]
        
    elif delta_Ti_2h_inp == 0 and post_Ta_avg_32h_inp == 0 and Tw_inp == 0 and Te_inp == 1:
    
        value_names = ["Ti", "Te", "pre Ta avg 12h", "post Ta avg 24h",
                       "pre SIso avg 4h", "post SIso avg 8h",  "pre Th avg 0.5h",
                       "hour with minutes", "delta Tset", "original"]

    elif delta_Ti_2h_inp == 0 and post_Ta_avg_32h_inp == 0 and Tw_inp == 1 and Te_inp == 0:
    
        value_names = ["Ti", "Tw", "pre Ta avg 12h", "post Ta avg 24h",
                       "pre SIso avg 4h", "post SIso avg 8h",  "pre Th avg 0.5h",
                       "hour with minutes", "delta Tset", "original"]
        
    elif delta_Ti_2h_inp == 0 and post_Ta_avg_32h_inp == 0 and Tw_inp == 0 and Te_inp == 0:
    
        value_names = ["Ti", "pre Ta avg 12h", "post Ta avg 24h",
                       "pre SIso avg 4h", "post SIso avg 8h",  "pre Th avg 0.5h",
                       "hour with minutes", "delta Tset", "original"]

    elif price_flag == 1 and no_future_inp == 0 and dif_price_inp == 0:
        
        value_names = ["Ti", "Tw", "Te", "price", "price min in x", "price max in x", "delta Ti 2h", "pre Ta avg 12h", "post Ta avg 24h",
                       "pre SIso avg 4h", "post SIso avg 8h", "post Ta avg 32h", "pre Th avg 0.5h",
                       "hour with minutes", "delta Tset", "original"]

    elif price_flag == 1 and no_future_inp == 1:
        
        value_names = ["Ti", "Tw", "Te", "price", "delta Ti 2h", "pre Ta avg 12h", "post Ta avg 24h",
                       "pre SIso avg 4h", "post SIso avg 8h", "post Ta avg 32h", "pre Th avg 0.5h",
                       "hour with minutes", "delta Tset", "original"]

    elif price_flag == 1 and dif_price_inp == 1:

        value_names = ["Ti", "Tw", "Te", "price", "delta price", "delta Ti 2h", "pre Ta avg 12h", "post Ta avg 24h",
                       "pre SIso avg 4h", "post SIso avg 8h", "post Ta avg 32h", "pre Th avg 0.5h",
                       "hour with minutes", "delta Tset", "original"]
    
   # Negate values in feat_results and calculate sums for average calculation
    sum_sensitivity = [0] * len(value_names)
    sum_nullify = [0] * len(value_names)
    for index, (sens, null) in enumerate(feat_results):
        negated_sens = [-x for x in sens]
        negated_null = [-x for x in null]
        feat_results[index] = (negated_sens, negated_null)
        sum_sensitivity = [sum_val + neg_val for sum_val, neg_val in zip(sum_sensitivity, negated_sens)]
        sum_nullify = [sum_val + neg_val for sum_val, neg_val in zip(sum_nullify, negated_null)]

    # Calculate averages
    avg_sensitivity = [val / len(feat_results) for val in sum_sensitivity]
    avg_nullify = [val / len(feat_results) for val in sum_nullify]

    # Add the average DataFrame so it can be treated like the others
    feat_results.append((avg_sensitivity, avg_nullify))

    # Speichern des 'original' Wertes für jede Population
    original_values = []

    for i, (sens, null) in enumerate(feat_results):
        # Speichern des 'original' Wertes
        original_sens = sens[-1]  # 'original' ist der erste Wert in der Liste
        original_null = null[-1]  # Gleiches gilt für Nullify
        original_values.append((original_sens, original_null))

        # Subtrahieren des 'original' Wertes von allen anderen Werten
        adjusted_sens = [x - original_sens for x in sens]
        adjusted_null = [x - original_null for x in null]
        adjusted_sens[0] = 0  # Setzen von 'original' auf 0
        adjusted_null[0] = 0  # Gleiches für Nullify
        feat_results[i] = (adjusted_sens, adjusted_null)

    # Initialize a list to store DataFrames for each population
    all_dataframes = []

    for i, (sens, null) in enumerate(feat_results, start=1):
        # Pair each feature name with its sensitivity and nullify values
        paired_sens = list(zip(value_names, sens))
        paired_null = list(zip(value_names, null))

        # Find and move 'original' to the first position
        original_sens_index = next(i for i, x in enumerate(paired_sens) if x[0] == 'original')
        original_null_index = next(i for i, x in enumerate(paired_null) if x[0] == 'original')
        original_sens = paired_sens.pop(original_sens_index)
        original_null = paired_null.pop(original_null_index)

        # Sort remaining features based on their values
        sorted_sens = sorted(paired_sens, key=lambda x: x[1], reverse=True)
        sorted_null = sorted(paired_null, key=lambda x: x[1], reverse=True)

        # Insert 'original' at the first position
        sorted_sens.insert(0, original_sens)
        sorted_null.insert(0, original_null)

        # Create separate lists for names and values
        names_sens, values_sens = zip(*sorted_sens)
        names_null, values_null = zip(*sorted_null)

        # Convert to DataFrame format
        df = pd.DataFrame({
            f'Pop Sens Name {i}': names_sens,
            f'Pop Sens Val {i}': values_sens,
            f'Pop Nullify Name {i}': names_null,
            f'Pop Nullify Val {i}': values_null
        })

        # Add the DataFrame to the list
        all_dataframes.append(df)

        # Add an empty DataFrame to create a separator column
        all_dataframes.append(pd.DataFrame({" ": ['']*len(df)}))

    # Combine all DataFrames into a single DataFrame
    combined_df = pd.concat(all_dataframes, axis=1)

    
    
    # Hinzufügen der 'old Original' Werte
    original_df = pd.DataFrame({
        'Name': ['old Original'] * len(original_values),
        'Sens Value': [val[0] for val in original_values],
        'Nullify Value': [val[1] for val in original_values]
    })
    combined_df = pd.concat([combined_df, pd.DataFrame({}), original_df], axis=1)

    
    # Bestimmen der maximalen Länge
    max_length = max(len(combined_df), len(test_scores), len(test_scores_index))

    # Leere Zeilen erstellen und an combined_df anhängen, falls notwendig
    if len(combined_df) < max_length:
        additional_rows = max_length - len(combined_df)
        # Erstellen eines DataFrames mit leeren Zeilen
        empty_df = pd.DataFrame([pd.Series()] * additional_rows)

        # Vor dem Zusammenführen der DataFrames die Indizes zurücksetzen
        combined_df.reset_index(drop=True, inplace=True)
        empty_df.reset_index(drop=True, inplace=True)

        # Jetzt die DataFrames zusammenführen
        combined_df = pd.concat([combined_df, empty_df], ignore_index=True)

            
    # Erstellen eines DataFrames für zusätzliche Informationen
    additional_info_df = pd.DataFrame({
        ' ': [' ', f'best_test_score= {best_test_score}', f'best_test= {best_test}'] + [''] * (max_length - 3),
        'test_scores': test_scores + [''] * (max_length - len(test_scores)),
        'test_scores_index': test_scores_index + [''] * (max_length - len(test_scores_index))
    })
    
    # Hinzufügen des zusätzlichen Info-DataFrames am Ende
    combined_df = pd.concat([combined_df, additional_info_df], axis=1)

    # Save the DataFrame to a CSV file
    combined_df.to_csv(save_path + "feat_result.csv", index=False)

    ### Feat Plot
    # Funktion zum Extrahieren von Daten für eine bestimmte Population
    def extract_pop_data(df, pop_num, data_type):
        return df[[f'Pop {data_type} Name {pop_num}', f'Pop {data_type} Val {pop_num}']]

    # Figur mit Subplots erstellen, vergrößere die Größe der Figur
    fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(35, 17))  # Größe angepasst

    # Iteration über jede Population, um einen Plot zu erstellen
    for i in range(5):
        # Sens-Daten extrahieren
        sens_data = extract_pop_data(combined_df, i + 1, "Sens")
        names_sens = sens_data[f'Pop Sens Name {i + 1}'].dropna()
        values_sens = sens_data[f'Pop Sens Val {i + 1}'].dropna()

        # Nullify-Daten extrahieren
        null_data = extract_pop_data(combined_df, i + 1, "Nullify")
        names_null = null_data[f'Pop Nullify Name {i + 1}'].dropna()
        values_null = null_data[f'Pop Nullify Val {i + 1}'].dropna()

        # Sens-Daten plotten
        axes[0, i].bar(names_sens, values_sens, alpha=0.6)
        axes[0, i].set_title(f'Population {i + 1} Sens')
        axes[0, i].set_xticks(range(len(names_sens)))
        axes[0, i].set_xticklabels(names_sens, rotation=45, ha='right')

        # Nullify-Daten plotten
        axes[1, i].bar(names_null, values_null, alpha=0.6)
        axes[1, i].set_title(f'Population {i + 1} Nullify')
        axes[1, i].set_xticks(range(len(names_null)))
        axes[1, i].set_xticklabels(names_null, rotation=45, ha='right')

    # Gesamttitel und Layout einstellen
    fig.suptitle('Feature Plots für Populationen (Sens und Nullify)', fontsize=20)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # Speichern des Plots als Bild
    plt.savefig(save_path + 'feat_plot.png')

    ### Test Plot
    # Erstellen des Plots
    plt.figure(figsize=(10, 6))
    plt.plot(test_scores_index, test_scores, label='Test Scores', color='blue')
    plt.xlabel('Iterationen')
    plt.ylabel('Test Scores')
    plt.title('Test Scores over Iterations')
    
    # Markieren des Punktes mit dem maximalen Wert
    max_score_index = test_scores.index(max(test_scores))
    plt.scatter(test_scores_index[max_score_index], test_scores[max_score_index], color='red', label='Maximaler Wert')
    
    # Legende hinzufügen
    plt.legend()
    
    # Speichern des Plots als Bild
    plt.savefig(save_path + 'test_scores_plot.png')

def create_new_input(x_seq_copy,t,next_result):
    
    # Calculate new features
    deltaTi_2h = x_seq_copy['Ti'].iloc[t+2] - x_seq_copy['Ti'].iloc[t+2-2*4]  # Ti_now - Ti_old
    Ta_avg_last12h = x_seq_copy['Ta'].iloc[t-12*4:t+2].mean()  # Average of next 12 hours
    Ta_avg_24h = x_seq_copy['Ta'].iloc[t:t+2+24*4].mean()  # Average of next 12 to 24 hours
    SIso_avg_4h = x_seq_copy['SIso'].iloc[t-4*2:t+2].mean()  # Average of next 4 hours
    SIso_avg_8h = x_seq_copy['SIso'].iloc[t+2:t+2+8*4].mean()  # Average of next 4 to 8 hours
    Ta_avg_next32h = x_seq_copy['Ta'].iloc[t+2:t+2+32*4].mean()  # Average of next 4 hours
    Th_avg_last2 = x_seq_copy['Th'].iloc[t-2:t].mean()  # Average of next 4 to 8 hours
    

    delta_Tset=0
    if x_seq_copy['Ti'].iloc[t+2] > max_t_eval:
        delta_Tset = max_t_eval - x_seq_copy['Ti'].iloc[t+2]
    elif x_seq_copy['Ti'].iloc[t+2] < min_t_eval:
        delta_Tset = x_seq_copy['Ti'].iloc[t+2] - min_t_eval

    # Extrahieren der Stunden und Minuten
    timestamp = x_seq_copy.index[t + 2]
    hours = timestamp.hour
    minutes = timestamp.minute
    hour_with_minutes = hours + minutes / 60.0

    input_ = 0
    norm_input = 0

    if delta_Ti_2h_inp == 1 and post_Ta_avg_32h_inp == 1 and Tw_inp == 1 and Te_inp == 1 and price_flag == 0:
        
        input_ = np.array([
            next_result.var['Ti'][-1], next_result.var['Tw'][-1], next_result.var['Te'][-1], 
            deltaTi_2h, Ta_avg_last12h, Ta_avg_24h, SIso_avg_4h, SIso_avg_8h, 
            Ta_avg_next32h, Th_avg_last2, hour_with_minutes, delta_Tset
        ]) 
                
        # Normalisiere Eingabedaten
        norm_input = np.array([val if param == 'deltaTi' or  param == 'delta_Tset' else normalize_data(val, min_max_values[param]['min'], min_max_values[param]['max']) for val, param in zip(input_, 
            ['Ti', 'Tw', 'Te', 'deltaTi', 'Ta_avg', 'Ta_avg', 'SIso_avg', 'SIso_avg', 'Ta_avg', 'Th', 'day_hours', 'delta_Tset'])])

    elif delta_Ti_2h_inp == 0 and post_Ta_avg_32h_inp == 0 and Tw_inp == 1 and Te_inp == 1:

        input_ = np.array([
            next_result.var['Ti'][-1], next_result.var['Tw'][-1], next_result.var['Te'][-1], 
            Ta_avg_last12h, Ta_avg_24h, SIso_avg_4h, SIso_avg_8h, 
            Th_avg_last2, hour_with_minutes, delta_Tset
        ]) 
                
        # Normalisiere Eingabedaten
        norm_input = np.array([val if param == 'deltaTi' or  param == 'delta_Tset' else normalize_data(val, min_max_values[param]['min'], min_max_values[param]['max']) for val, param in zip(input_, 
            ['Ti', 'Tw', 'Te',  'Ta_avg', 'Ta_avg', 'SIso_avg', 'SIso_avg', 'Th', 'day_hours', 'delta_Tset'])])

    elif delta_Ti_2h_inp == 0 and post_Ta_avg_32h_inp == 0 and Tw_inp == 0 and Te_inp == 1:

        input_ = np.array([
            next_result.var['Ti'][-1], next_result.var['Te'][-1], 
            Ta_avg_last12h, Ta_avg_24h, SIso_avg_4h, SIso_avg_8h, 
            Th_avg_last2, hour_with_minutes, delta_Tset
        ]) 
                
        # Normalisiere Eingabedaten
        norm_input = np.array([val if param == 'deltaTi' or  param == 'delta_Tset' else normalize_data(val, min_max_values[param]['min'], min_max_values[param]['max']) for val, param in zip(input_, 
            ['Ti', 'Te', 'Ta_avg', 'Ta_avg', 'SIso_avg', 'SIso_avg', 'Th', 'day_hours', 'delta_Tset'])])

    elif delta_Ti_2h_inp == 0 and post_Ta_avg_32h_inp == 0 and Tw_inp == 1 and Te_inp == 0:

        input_ = np.array([
            next_result.var['Ti'][-1], next_result.var['Tw'][-1],
            Ta_avg_last12h, Ta_avg_24h, SIso_avg_4h, SIso_avg_8h, 
            Th_avg_last2, hour_with_minutes, delta_Tset
        ]) 
                
        # Normalisiere Eingabedaten
        norm_input = np.array([val if param == 'deltaTi' or  param == 'delta_Tset' else normalize_data(val, min_max_values[param]['min'], min_max_values[param]['max']) for val, param in zip(input_, 
            ['Ti', 'Tw',  'Ta_avg', 'Ta_avg', 'SIso_avg', 'SIso_avg', 'Th', 'day_hours', 'delta_Tset'])])

    elif delta_Ti_2h_inp == 0 and post_Ta_avg_32h_inp == 0 and Tw_inp == 0 and Te_inp == 0:

        input_ = np.array([
            next_result.var['Ti'][-1], 
            Ta_avg_last12h, Ta_avg_24h, SIso_avg_4h, SIso_avg_8h, 
            Th_avg_last2, hour_with_minutes, delta_Tset
        ]) 
                
        # Normalisiere Eingabedaten
        norm_input = np.array([val if param == 'deltaTi' or  param == 'delta_Tset' else normalize_data(val, min_max_values[param]['min'], min_max_values[param]['max']) for val, param in zip(input_, 
            ['Ti', 'Ta_avg', 'Ta_avg', 'SIso_avg', 'SIso_avg', 'Th', 'day_hours', 'delta_Tset'])])


    elif price_flag == 1 and no_future_inp == 0 and dif_price_inp == 0:
        input_ = np.array([
            next_result.var['Ti'][-1], next_result.var['Tw'][-1], next_result.var['Te'][-1], x_seq_copy['price'].iloc[t+2], x_seq_copy['price_min_in_x'].iloc[t+2], x_seq_copy['price_min_in_x'].iloc[t+2],         
            deltaTi_2h, Ta_avg_last12h, Ta_avg_24h, SIso_avg_4h, SIso_avg_8h, 
            Ta_avg_next32h, Th_avg_last2, hour_with_minutes, delta_Tset
        ]) 
                
        # Normalisiere Eingabedaten
        norm_input = np.array([val if param == 'deltaTi' or  param == 'delta_Tset' else normalize_data(val, min_max_values[param]['min'], min_max_values[param]['max']) for val, param in zip(input_, 
            ['Ti', 'Tw', 'Te', 'price', 'price', 'price', 'deltaTi', 'Ta_avg', 'Ta_avg', 'SIso_avg', 'SIso_avg', 'Ta_avg', 'Th', 'day_hours', 'delta_Tset'])])

    elif price_flag == 1 and dif_price_inp == 1:
        input_ = np.array([
            next_result.var['Ti'][-1], next_result.var['Tw'][-1], next_result.var['Te'][-1], x_seq_copy['price'].iloc[t+2], x_seq_copy['price_delta'].iloc[t+2],         
            deltaTi_2h, Ta_avg_last12h, Ta_avg_24h, SIso_avg_4h, SIso_avg_8h, 
            Ta_avg_next32h, Th_avg_last2, hour_with_minutes, delta_Tset
        ]) 
                
        # Normalisiere Eingabedaten
        norm_input = np.array([val if param == 'deltaTi' or  param == 'delta_Tset' or  param == 'delta_price' else normalize_data(val, min_max_values[param]['min'], min_max_values[param]['max']) for val, param in zip(input_, 
            ['Ti', 'Tw', 'Te', 'price', 'deltaTi', 'Ta_avg', 'Ta_avg', 'SIso_avg', 'SIso_avg', 'Ta_avg', 'Th', 'day_hours', 'delta_Tset'])])

    
    elif price_flag == 1 and no_future_inp == 1:
        input_ = np.array([
            next_result.var['Ti'][-1], next_result.var['Tw'][-1], next_result.var['Te'][-1], x_seq_copy['price'].iloc[t+2],      
            deltaTi_2h, Ta_avg_last12h, Ta_avg_24h, SIso_avg_4h, SIso_avg_8h, 
            Ta_avg_next32h, Th_avg_last2, hour_with_minutes, delta_Tset
        ]) 
                
        # Normalisiere Eingabedaten
        norm_input = np.array([val if param == 'deltaTi' or  param == 'delta_Tset' else normalize_data(val, min_max_values[param]['min'], min_max_values[param]['max']) for val, param in zip(input_, 
            ['Ti', 'Tw', 'Te', 'price', 'deltaTi', 'Ta_avg', 'Ta_avg', 'SIso_avg', 'SIso_avg', 'Ta_avg', 'Th', 'day_hours', 'delta_Tset'])])

    
    return norm_input, hour_with_minutes, timestamp



def feature_importance_test(sens_or_null,start_idx_w_pre, pop_idx,input_num,pre_sequence_length,post_sequence_length,x_seq,t,input_layer,
                                        weights_biases,net_params):
    feat_result = []
    
    for i_input in range(input_num+1):
        
        x_seq_copy_fit = x_seq
        
        reward_X_days_fit = []
        energy_X_days_fit = []
        distance_X_days_fit = []

        bkt_state_last_fit = 1
        
        for t in range(pre_sequence_length, steps_X_days - post_sequence_length, 2):

            if t == pre_sequence_length:  
                bkt_state_last_fit = 1

                ic_params_map_new = {
                    'Ti0': lambda X, y, model_result: result_TRAIN.var['Ti'][start_idx_w_pre],
                    'Tw0': lambda X, y, model_result: result_TRAIN.var['Tw'][start_idx_w_pre],
                    'Te0': lambda X, y, model_result: result_TRAIN.var['Te'][start_idx_w_pre],
                }
            else:
                ic_params_map_new = {
                    'Ti0': lambda X, y, model_result: next_result_fit.var['Ti'][-1],
                    'Tw0': lambda X, y, model_result: next_result_fit.var['Tw'][-1],
                    'Te0': lambda X, y, model_result: next_result_fit.var['Te'][-1],
                }

            next_predict = predict_model(model_η_train, x_seq_copy_fit.iloc[t:t+2], y_seq.iloc[t:t+2], ic_params_map_new, error_metric, model_η_train.result)
            next_result_fit = next_predict['model_result'].iloc[0]

            norm_input_fit, hour_with_minutes_fit, timestamp_fit = create_new_input(x_seq_copy_fit,t,next_result_fit)
            
            if sens_or_null == True:
                # Permutate:
                zufalls_permutation = random.random()
                if i_input != input_num:
                    norm_input_fit[i_input] = zufalls_permutation
            else: 
                # Nullify
                if i_input != input_num:
                    norm_input_fit[i_input] = 0
            
            input_layer = norm_input_fit.reshape(1, input_num)
            normed_nn_out = forward_path_nnr(input_layer, weights_biases, net_params)

            bkt_vorlauftemp = 0
            # Denormalisiere NN-Ausgabedaten
            if num_outputs == 1:
                bkt_vorlauftemp = denormalize_data(normed_nn_out[0], min_max_values['Th']['min'], min_max_values['Th']['max'])
            else:
                bkt_vorlauftemp = denormalize_data(normed_nn_out[0][0], min_max_values['Th']['min'], min_max_values['Th']['max'])

            bkt_vorlauftemp_15, bkt_vorlauftemp_30 = calculate_bkt_vorlauftemo(normed_nn_out,x_seq_copy_fit,t,next_result_fit)

            bkt_state_last = bkt_an 
            
            timestamp_t_plus_15 = x_seq_copy_fit.index[t] + timedelta(minutes=15)
            timestamp_t_plus_30 = x_seq_copy_fit.index[t] + timedelta(minutes=30)
            x_seq_copy_fit.at[timestamp_t_plus_15, 'Th'] = bkt_vorlauftemp_15
            x_seq_copy_fit.at[timestamp_t_plus_30, 'Th'] = bkt_vorlauftemp_30

            price_now = x_seq_copy_fit.at[timestamp_t_plus_15, 'price']
            
            new_Ti = next_result_fit.var['Ti'][-1]

            # Evaluate Step
            if t > (pre_sequence_length+ignor_first_half_day):
                energy_taken = evaluate_energy(bkt_an,bkt_vorlauftemp_30,next_result.var['Tw'][-1],bkt_state_last, price_now) 
                e2_distance = evaluate_vorlauf_range(bkt_vorlauftemp) 
                e2_to_goal_temp = evaluate_temp(new_Ti,hour_with_minutes_fit)
                reward_step = e2_to_goal_temp + e2_distance + energy_taken

                energy_X_days_fit.append(energy_taken)
                distance_X_days_fit.append(e2_distance)
                abweichung_temp_X_days.append(e2_to_goal_temp)
                reward_X_days_fit.append(reward_step)

        # Evaluate example
        reward_avg_permut_score_fit = np.mean(reward_X_days_fit)
        feat_result.append(reward_avg_permut_score_fit)
    
    return feat_result



def validate_model(df, seq, seq_len, model, error_metric, same_start_time, start_time, predict_function, return_output=True, show_plot=False):
    
    """
    df: DataFrame with the test dataset
    seq: Number of sequences to test
    seq_len: Length of each sequence in days
    params: Model parameters (a, b, c, d)
    same_start_time: If True, all sequences will start at the same time of day
    start_time: Desired start time in hours for the sequences
    """
    
     # Calculate the sequence length in rows
    sequence_length = int(seq_len * 24 * 4)  # seq_len days, 24 hours a day, 4 quarters of an hour per hour
    
    # Calculate the number of rows for the 6 hours pre-sequence
    pre_sequence_length = 6 * 4
    
    if same_start_time:
        # Calculate the maximum number of distinct sequences with the same start time
        num_distinct_days = len(df) // (24 * 4)
        max_sequences = min(num_distinct_days, seq)
        
        if seq > max_sequences:
            warnings.warn(f"Requested number of sequences {seq} exceeds the maximum "
                          f"distinct sequences possible {max_sequences}. Using {max_sequences} sequences.")
            seq = max_sequences
        
        # Calculate the start indices for the sequences considering the start_time
        day_interval = num_distinct_days // seq
        start_indices = [(i * day_interval * 24 * 4) + (start_time * 4) - pre_sequence_length for i in range(seq)]
    else:
        # Calculate the start indices for the sequences considering the start_time
        interval = (len(df) - sequence_length) // seq
        start_indices = [(i * interval) + (start_time * 4) - pre_sequence_length for i in range(seq)]
    
    # Ensure the start indices are within the bounds of the DataFrame
    start_indices = [max(index, 0) for index in start_indices]
    
    # Initialize a list to store the errors for each time step
    errors = []
    
    for start_index in start_indices:
        # Extract the sequence with pre-sequence from the DataFrame
        sequence_df = df.iloc[start_index:start_index + sequence_length + pre_sequence_length].copy()
        
        # Calculate the weighted average for Te0 using the pre-sequence
        weights = np.arange(1, pre_sequence_length + 1)
        Agw = np.average(sequence_df.iloc[:pre_sequence_length]['Ta'], weights=weights)
        Igw = np.average(sequence_df.iloc[:pre_sequence_length]['Ti'], weights=weights)
        Te0 = Agw * 0.2 + Igw * 0.8
        
        # Update the initial condition for Te0 in the sequence
        sequence_df.iloc[pre_sequence_length]['Te0'] = Te0
        
        # Extract the actual sequence for prediction after calculating Te0
        sequence_df = sequence_df.iloc[pre_sequence_length:]
        
        # Call the abstract predict function to get the predictions
        predictions = predict_function(sequence_df, model, error_metric, Te0)
        
        # Calculate the error for each time step and append it to the list
        errors.append(np.abs(predictions - sequence_df['Ti'].to_numpy()))
    
    # Calculate the average error at each time step
    average_errors = np.mean(errors, axis=0)
    
    # Calculate the standard deviation of the error at each time step
    std_errors = np.std(errors, axis=0)
    
    if show_plot == True:
        # Plot the average errors with standard deviation
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(0, sequence_length) / 4, average_errors, label='Average Error')
        plt.fill_between(np.arange(0, sequence_length) / 4, average_errors - std_errors, average_errors + std_errors, alpha=0.2, label='Standard Deviation')
        plt.xlabel('Time (hours)')
        plt.ylabel('Error (°C)')
        plt.title('Average Error of Predictions at Each Time Step with Standard Deviation')
        plt.xticks(np.arange(0, sequence_length / 4 + 1, 6))
        plt.legend()
        plt.show()
    
    # Find the first index where the average error exceeds 1°C and convert it to hours
    first_exceed_index = np.argmax(average_errors > 0.4)
    first_exceed_time = first_exceed_index / 4

    # Print the result
    print(f"The first prediction time where the average error exceeds 0.4°C is at {first_exceed_time} hours.")
    
    # Create a DataFrame with the average_errors and std_errors
    results_df = pd.DataFrame({
        'average_errors': average_errors,
        'std_errors': std_errors
    })

    # Save the DataFrame to a CSV file
    results_df.to_csv('model_RC_results.csv', index=False)
    
    if return_output == True: 
        return errors
    

def calculate_weighted_means(pre_sequence_df):
    # Berechne die gewichteten Mittelwerte von Agw und Igw
    # Hier verwenden wir das DataFrame pre_sequence_df, das die Daten der 6 Stunden vor der Sequenz enthält
    
    # Gewichtungsfaktoren für die zeitliche Gewichtung, wobei die neueren Werte stärker gewichtet werden
    weights = np.linspace(1, 2, len(pre_sequence_df))
    
    # Gewichteter Mittelwert von Agw
    Agw = np.average(pre_sequence_df['Ta'], weights=weights)
    
    # Gewichteter Mittelwert von Igw (hier nehme ich an, dass 'Ti' die Innentemperatur ist)
    Igw = np.average(pre_sequence_df['Ti'], weights=weights)
    
    return Agw, Igw


def calculate_start_indices(df, seq, seq_len, same_start_time, start_time):
    sequence_length = int(seq_len * 24 * 4)
    
    if same_start_time:
        num_distinct_days = len(df) // (24 * 4)
        max_sequences = min(num_distinct_days, seq)
        if seq > max_sequences:
            seq = max_sequences
        day_interval = num_distinct_days // seq
        start_indices = [(i * day_interval * 24 * 4) + (start_time * 4) for i in range(seq)]
    else:
        interval = (len(df) - sequence_length) // seq
        start_indices = [(i * interval) + (start_time * 4) for i in range(seq)]
    
    return start_indices


def extract_random_sequence_with_seed(df, series, length, seed):
   
    # Stelle sicher, dass die Länge nicht größer als die kleinste Länge von df und series ist
    min_length = min(len(df), len(series))
    if length > min_length:
        raise ValueError(f"Die angeforderte Länge {length} ist größer als die minimale Länge {min_length} von DataFrame oder Series.")

    # Initialisiere den Zufallszahlengenerator
    np.random.seed(seed)

    # Finde gemeinsame Zeitstempel
    common_timestamps = df.index.intersection(series.index)

    # Wähle einen zufälligen Startpunkt
    start_idx = np.random.choice(len(common_timestamps) - length + 1)
    end_idx = start_idx + length

    # Extrahiere die Sequenzen
    selected_timestamps = common_timestamps[start_idx:end_idx]
    df_seq = df.loc[selected_timestamps]
    series_seq = series.loc[selected_timestamps]

    return df_seq, series_seq, start_idx, end_idx

def cut_data(df, series, anteil_train):
    # Stelle sicher, dass der Anteil für die Trainingsdaten sinnvoll ist
    if not (0 < anteil_train < 100):
        raise ValueError(f"Der angegebene Anteil für Trainingsdaten {anteil_train} ist nicht im Bereich von 0 bis 100.")

    # Finde gemeinsame Zeitstempel
    common_timestamps = df.index.intersection(series.index)

    # Berechne die Längen für Trainings- und Testdaten
    total_length = len(common_timestamps)
    length_train = int(total_length * anteil_train / 100)
    length_test = total_length - length_train

    # Extrahiere die Sequenzen für Trainingsdaten
    start_idx_train = 0
    end_idx_train = start_idx_train + length_train
    selected_timestamps_train = common_timestamps[start_idx_train:end_idx_train]
    df_seq_train = df.loc[selected_timestamps_train]
    series_seq_train = series.loc[selected_timestamps_train]

    # Extrahiere die Sequenzen für Testdaten
    start_idx_test = end_idx_train
    end_idx_test = start_idx_test + length_test
    selected_timestamps_test = common_timestamps[start_idx_test:end_idx_test]
    df_seq_test = df.loc[selected_timestamps_test]
    series_seq_test = series.loc[selected_timestamps_test]

    return df_seq_train, series_seq_train, start_idx_train, end_idx_train, df_seq_test, series_seq_test, start_idx_test, end_idx_test






#################################### Load and Preprocess Data

# Load ZSW-Data dataset
data_path = './data/ZSW_Data_Full_2023-08-09_2023-09-26.csv'
data_path_price = './data/day-ahead_price_quarter_hourly.csv'
input_df = pd.read_csv(data_path, index_col=0, parse_dates=True)

# Rename columns according to the mapping provided
column_mapping = {
    'ZSW_avg_Avg': 'Ti', #oder "Schacht A_Avg" oder "Schacht B_Avg"
    'Tamb': 'Ta',
    'BKT Temperatur': 'Th',
    'SI_45': 'SIno',
    'SI_135': 'SIso',
    'Klimatisierung An/Aus':'BKT_an'
}

input_df.rename(columns=column_mapping, inplace=True)

# Add additional columns 'Ti0', 'Te0', and 'Tw0'
input_df['Ti0'] = input_df['Ti']
input_df['Tw0'] = input_df['Ti']  
input_df['Te0'] = input_df['Ti'] 

# Remove other columns
columns_to_keep = ['Th', 'Ta', 'SIno', 'SIso', 'Ti0', 'Te0', 'Tw0', 'Ti','BKT_an']
input_df = input_df[columns_to_keep]

# Check which columns have missing values
missing_columns = input_df.columns[input_df.isna().any()].tolist()

# Separate data into daily segments
daily_segments = [group for name, group in input_df.groupby(input_df.index.date)]

# Perform spline interpolation within each segment
interpolated_segments = []
for segment in daily_segments:
    if len(segment) >= 4:  # Ensure segment has enough data points for cubic spline interpolation
        interpolated_segment = segment.interpolate(method='spline', order=3)
    else:
        interpolated_segment = segment.interpolate(method='linear')  # Use linear interpolation for small segments
    interpolated_segments.append(interpolated_segment)

# Combine the segments back together
interpolated_df = pd.concat(interpolated_segments)

# Fill any remaining missing values at the edges
interpolated_df = interpolated_df.ffill().bfill()

# Update the input_df with interpolated values
input_df[missing_columns] = interpolated_df[missing_columns]



# Update the input_df with interpolated values
input_df[missing_columns] = interpolated_df[missing_columns]

# Laden der day-ahead_price_quarter_hourly.csv-Datei
day_ahead_data = pd.read_csv(data_path_price, index_col='datetime', parse_dates=True)

# Konvertieren von Zeitstempeln in day_ahead_data in tz-naive (wenn sie tz-aware sind)
if day_ahead_data.index.tz is not None:
    day_ahead_data.index = day_ahead_data.index.tz_localize(None)

# Konvertieren von Zeitstempeln in input_df in tz-naive (wenn sie tz-aware sind)
if input_df.index.tz is not None:
    input_df.index = input_df.index.tz_localize(None)

# Zusammenführen der Daten basierend auf dem Index (Datum und Uhrzeit)
# Hierbei wird die Spalte aus day_ahead_data direkt als 'day_ahead_price' in input_df eingefügt
input_df = input_df.merge(day_ahead_data['day_ahead_auction'].rename('price'), left_index=True, right_index=True, how='left')

def add_min_max_future_steps(df, look_future_steps):
    # Berechnen des minimalen Werts in den nächsten 'look_future_steps' Schritten
    df[f'price_min_in_x'] = df['price'].rolling(window=look_future_steps, min_periods=1).min().shift(-look_future_steps + 1)

    # Berechnen des maximalen Werts in den nächsten 'look_future_steps' Schritten
    df[f'price_max_in_x'] = df['price'].rolling(window=look_future_steps, min_periods=1).max().shift(-look_future_steps + 1)

    df['price_min_in_x'] = df['price_min_in_x'].ffill()
    df['price_max_in_x'] = df['price_max_in_x'].ffill()
    
    return df

# Beispielaufruf der Funktion
# Ersetzen Sie dies durch die gewünschte Anzahl an zukünftigen Schritten
input_df = add_min_max_future_steps(input_df, look_future_steps)

# Umwandeln der 'price'-Spalte in einen Float-Typ
def convert_columns_to_numeric(df, column_names):
    for column in column_names:
        if column in df.columns:
            df[column] = pd.to_numeric(df[column], errors='coerce')

# Konvertieren der relevanten Spalten in numerische Werte
columns_to_convert = ['price', 'price_min_in_x', 'price_max_in_x']
convert_columns_to_numeric(input_df, columns_to_convert)

# Extract input features and target variable
input_X = input_df[['Th', 'Ta', 'SIno', 'SIso', 'Ti0', 'Te0', 'Tw0', 'price', 'price_min_in_x', 'price_max_in_x']]
input_y = input_df['Ti']

# Print shapes of input features and target variable
#print(f'Input X shape: {input_X.shape}, input y shape: {input_y.shape}')

# The duration of a record
rec_duration = 0.25  # hour





#################################### Create Train/Test Data

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(input_X, input_y, test_size=5 / 33, shuffle=False)

# Print shapes of training and test sets
#print(f'Train: X shape: {X_train.shape}, y shape: {y_train.shape}')
#print(f'Test: X shape: {X_test.shape}, y shape: {y_test.shape}')

# Check for missing values (NaN) in the dataset
nan_counts = input_df.isna().sum()
nan_counts[nan_counts > 0]

#print("")
#print(nan_counts)
#print("")
#print(input_df)

#plt.show()

#plot_input_data(input_df, with_Kh = False, with_Ph = False)



#################################### Erzeuge oder Lade RC-Modell und Modell configs and Splits

train_params_TiTeTwSInoSIso = {
    'Ti0': {'value': 35, 'vary': False},
    'Te0': {'value': 10, 'vary': False},
    'Tw0': {'value': 30, 'vary': False},    
    'Ci': {'value': 12.41119259, 'min': 0.1, 'max': 400},
    'Ce': {'value': 436.080547 ,'min': 0.2, 'max': 2000},
    'Cw': {'value': 170.128391 ,'min': 0.2, 'max': 2000},
    'Rie': {'value': 2.60363635, 'min': 0.001, 'max': 500},
    'Rea': {'value': 4.73124347, 'min': 0.001, 'max': 500},
    'Riw': {'value': 3.61742182, 'min': 0.001, 'max': 500},
    'Rhw': {'value': 0.40619, 'min': 0.001, 'max': 500},
    'Aino': {'value': 0.00279975, 'min': 0.0000001, 'max': 30},
    'Aiso': {'value': 0.00494192, 'min': 0.0000001, 'max': 30},
}

ic_params_map = {
    'Ti0': lambda X_test, y_test, train_result: y_train.iloc[0],
    'Tw0': lambda X_test, y_test, train_result: y_train.iloc[0],
    'Te0': lambda X_test, y_test, train_result: train_result.var['Te'][-1],
}

models = [
          #TiTeTh(train_params_TiTeTh, rec_duration),
          TiTeTwSInoSIso(train_params_TiTeTwSInoSIso, rec_duration),
          #TiTeThSIη(train_params_TiTeThSIη, rec_duration),
        ]

prefit_splits = custom_splits(X_train, num_sets=1, train_len=60, val_len=20, variance=70)
#splits_list = list(prefit_splits)

#prefit_splits = KFold(n_splits=int(len(X_train) / 96), shuffle=False).split(X_train)

error_metric = rmse

prefit_filter = lambda error: abs(error) < 10

method = 'nelder'
#method = 'leastsq'

#################################### Train RC-Modell

df = darkgreyfit(models, X_train, y_train, X_test, y_test, ic_params_map, error_metric,
                 prefit_splits=prefit_splits, prefit_filter=prefit_filter, reduce_train_results=True, 
                 method=method, n_jobs=-1, verbose=10)


#################################### Select Modell

pos = 0

# Example usage: selecting the model at position 1
model_η_train = select_model_by_position(df, pos)

train_results_η_train = df.loc[pos, ('train', 'model_result')]
test_results_η_train = df.loc[pos, ('test', 'model_result')]

# model_η_train.result.params

#################################### Plot Modell Simulation Results

#plot_with_Th(y_train, train_results_η_train, 'TiTeThSIη (Train)', X_with_out=None, with_Tw=True)
#plot_with_Th(y_train, train_results_η_train, 'TiTeThSIη (Train)', X_with_out=X_train, with_Th=True, with_Ta=True, with_SIno=True, with_SIso=True, with_Tw=True)
#plot_with_Th(y_test, test_results_η_train, 'TiTeThSIη (Train)', X_with_out=None, with_Tw=True)
#plot_with_Th(y_test, test_results_η_train, 'TiTeThSIη (Train)', X_with_out=X_test,with_Th=True, with_Ta=True, with_SIno=True, with_SIso=True, with_Tw=True)

#################################### Override Modell to load old values

model_η_train.result.params['Ci'].value = 27.8315427
model_η_train.result.params['Ce'].value = 585.492669
model_η_train.result.params['Cw'].value = 132.201565
model_η_train.result.params['Rie'].value = 0.93187567
model_η_train.result.params['Rea'].value = 0.46359172
model_η_train.result.params['Riw'].value = 0.37819626
model_η_train.result.params['Rhw'].value = 0.67818813
model_η_train.result.params['Aino'].value = 0.00899538
model_η_train.result.params['Aiso'].value = 0.01125552


#################################### Make Prediction for Data

# Split the data into training and test sets
X_TRAIN, X_TEST, y_TRAIN, y_TEST = train_test_split(input_X, input_y, test_size = 5 / 33, shuffle=False)

ic_params_map_start = {
    'Ti0': lambda X, y, model_result: y_TRAIN.iloc[0],
    'Tw0': lambda X, y, model_result: y_TRAIN.iloc[0],
    'Te0': lambda X, y, model_result: y_TRAIN.iloc[0],
}

prediction_TRAIN = predict_model(model_η_train, X_TRAIN, y_TRAIN, ic_params_map_start, error_metric, model_η_train.result)

result_TRAIN = prediction_TRAIN['model_result'].iloc[0]  

#plot_with_Th(y_TRAIN, result_TRAIN, 'TiTeThSIη (Train)', X_with_out=None, with_Tw=True)






##################################################################################################################################

######################################################## NN-Controller ###########################################################

##################################################################################################################################



# Netzwerkstruktur festlegen
structure = [input_num, num_hidden_neurons, num_outputs]  # Eingabe hat 2 Merkmale, eine versteckte Schicht mit 4 Neuronen und eine Ausgabeschicht mit 1 Neuron

# Anzahl der Parameter berechnen und PGPE-Optimierer initialisieren
num_paras = get_num_paras_from_structure(structure)
para_dict = OP()

for i in range(num_paras):
    para_dict.add_parameter(
        key=f"param_{i}",
        min=-10.0, max=10.0, type="f", scale="linear", init=0.0
    )

np.random.seed(169)
start_seed = np.random.randint(0, 10000)
initial_params = np.array([para_dict.get_init_parameters_array()])
seed_int = int(np.sum(initial_params))
para_dict = para_dict.parameter_dict
optimizer = RankSuperSymPGPE(para_dict, plot_paras=False, seed=initial_params)

optimizer.mAlpha = mue_alpha
optimizer.sAlpha = sigma_alpha

# Define the PT1 function
def use_pt1_function(t, beharrungswert, start_temp, T = 1506.6743): 
    return beharrungswert - (beharrungswert - start_temp) * np.exp(-t / T)

#Grenzen des Heiz- und Kühlfalls:
kühl_temp_min = 17.5
kühl_temp_max = 20.5
heiz_temp_min = 22
heiz_temp_max = 28

#Arbeitszeiten definieren
start_working_hour = 6
end_working_hour = 20
    
# Define a manual color scheme for the plots
color_scheme = {
    'Ti': 'blue', 'Th': 'orange', 'Tw': 'green',
    'Ta': 'red', 'Te': 'purple', 'SIno': 'brown', 'SIso': 'pink', 
    'price': 'olive','price_min_in_x': 'navy','price_max_in_x': 'maroon'
}

X_TRAIN.rename(columns={'Ti0': 'Ti'}, inplace=True)

save_path = new_folder_path ##'./XXXX/ExampleRun/'
num_samples = num_iterations

#Init Variables
steps_X_days = 0
ignor_first_half_day = 0
take_test_data = False
avg_reward_seq_model = 0
last_Iteration = False
best_test = -999
best_test_score = -999.9
test_scores = []
test_scores_index = []
result_TRAIN_copy = result_TRAIN

# Initialize dictionaries to store the reward history
reward_history = {pop: [] for pop in range(optimizer.pop_size)}
model_reward_history = []
final_weights_biases = []


print("#####################################")
print("###########  Starting RL  ###########")


for i in range(num_samples):
    
    print("")
    print(f" ########### I: {i} ############")
    
    if i % take_test_data_every == 0 and i != 0:
        take_test_data = True
    else: 
        take_test_data = False
    
    if i == (num_samples-1):
        last_Iteration = True
    
    if i % make_plot_each == 0 or take_test_data == True:
        modell_results_dict = {pop: [] for pop in range(optimizer.pop_size)}
    
    # Adjust sequence extraction to include pre- and post-sequences
    x_seq, y_seq, start_idx, end_idx, x_seq_test, y_seq_test, start_idx_test, end_idx_test = cut_data(df=X_TRAIN, series=y_TRAIN, anteil_train=85)
    
    if take_test_data == True:
        start_idx = start_idx_test
        end_idx = end_idx_test
        x_seq = x_seq_test
        y_seq = y_seq_test
    
    steps_X_days = end_idx - start_idx
    pre_sequence_length = 12*4  # 2 hours back, 4 timesteps per hour
    post_sequence_length = 32*4+2  # 24 hours forward, 4 timesteps per hour
    
    # Adjust start_idx and end_idx to account for pre-sequence
    start_idx_w_pre = start_idx + pre_sequence_length
    end_idx_w_post = end_idx - post_sequence_length
    
    population = optimizer.ask()
    rewards = []
    reward_avgs = []
    avgs_energy_X_days = []
    avgs_distance_X_days = []
    avgs_abweichung_temp_X_days = []
    
    bkt_an = 1
    bkt_state_last = 1
    
    feat_results = []
    
    for pop_idx, para_vector in enumerate(population):
        
        x_seq_copy = x_seq
        weights_biases, net_params = make_weights_biases_from_vector(para_vector, structure)
        
        reward_X_days = []
        energy_X_days = []
        distance_X_days = []
        abweichung_temp_X_days = []
        
        # Feature Importance
        feat_importance_results = []
        if (last_Iteration == True or i % update_feat_every == 0) and i > no_feat_imp_bevore:
            final_weights_biases.append(weights_biases)
            
            sens_feat_results = feature_importance_test(True,start_idx_w_pre, pop_idx,input_num,pre_sequence_length,post_sequence_length,x_seq_copy,t,bkt_state_last,input_layer,
                                    weights_biases,net_params)
            null_feat_results = feature_importance_test(False,start_idx_w_pre, pop_idx,input_num,pre_sequence_length,post_sequence_length,x_seq_copy,t,bkt_state_last,input_layer,
                                    weights_biases,net_params)
         
            feat_results.append([sens_feat_results,null_feat_results])
        
        for t in range(pre_sequence_length, steps_X_days - post_sequence_length, 2):
            
            if t == pre_sequence_length:  
                bkt_state_last = 1
                
                ic_params_map_new = {
                    'Ti0': lambda X, y, model_result: result_TRAIN.var['Ti'][start_idx_w_pre],
                    'Tw0': lambda X, y, model_result: result_TRAIN.var['Tw'][start_idx_w_pre],
                    'Te0': lambda X, y, model_result: result_TRAIN.var['Te'][start_idx_w_pre],
                }
            else:
                ic_params_map_new = {
                    'Ti0': lambda X, y, model_result: next_result.var['Ti'][-1],
                    'Tw0': lambda X, y, model_result: next_result.var['Tw'][-1],
                    'Te0': lambda X, y, model_result: next_result.var['Te'][-1],
                }

            next_predict = predict_model(model_η_train, x_seq_copy.iloc[t:t+2], y_seq.iloc[t:t+2], ic_params_map_new, error_metric, model_η_train.result)
            next_result = next_predict['model_result'].iloc[0]
            
            norm_input, hour_with_minutes, timestamp = create_new_input(x_seq_copy,t,next_result)
            
            input_layer = norm_input.reshape(1, input_num)
            normed_nn_out = forward_path_nnr(input_layer, weights_biases, net_params)
            
            bkt_vorlauftemp = 0
            
            # Denormalisiere NN-Ausgabedaten
            if num_outputs == 1:
                bkt_vorlauftemp = denormalize_data(normed_nn_out[0], min_max_values['Th']['min'], min_max_values['Th']['max'])
            else:
                bkt_vorlauftemp = denormalize_data(normed_nn_out[0][0], min_max_values['Th']['min'], min_max_values['Th']['max'])
                
            bkt_vorlauftemp_15, bkt_vorlauftemp_30 = calculate_bkt_vorlauftemo(normed_nn_out,x_seq_copy,t,next_result)
            
            bkt_state_last = bkt_an 
            
            timestamp_t_plus_15 = x_seq_copy.index[t] + timedelta(minutes=15)
            timestamp_t_plus_30 = x_seq_copy.index[t] + timedelta(minutes=30)
            x_seq_copy.at[timestamp_t_plus_15, 'Th'] = bkt_vorlauftemp_15
            x_seq_copy.at[timestamp_t_plus_30, 'Th'] = bkt_vorlauftemp_30
            
            new_Ti = next_result.var['Ti'][-1]
            
            if i % make_plot_each == 0 or take_test_data == True:
                
                timestamp = x_seq_copy.index[t+2]
                new_row = 0
                if price_flag == 1:
                    if no_future_inp == 1:
                        new_row = { 'Timestamp': timestamp, 'Ti': next_result.var['Ti'][-1], 'Tw': next_result.var['Tw'][-1], 'Te': next_result.var['Te'][-1], 'Ta': x_seq_copy['Ta'].iloc[t+2], 'Th': x_seq_copy['Th'].iloc[t+2],'price':x_seq_copy['price'].iloc[t+2]}
                    else:
                        new_row = { 'Timestamp': timestamp, 'Ti': next_result.var['Ti'][-1], 'Tw': next_result.var['Tw'][-1], 'Te': next_result.var['Te'][-1], 'Ta': x_seq_copy['Ta'].iloc[t+2], 'Th': x_seq_copy['Th'].iloc[t+2],'price':x_seq_copy['price'].iloc[t+2],'price_min_in_x': x_seq_copy['price_min_in_x'].iloc[t+2], 'price_max_in_x':x_seq_copy['price_max_in_x'].iloc[t+2] }
                else:
                    new_row = { 'Timestamp': timestamp, 'Ti': next_result.var['Ti'][-1], 'Tw': next_result.var['Tw'][-1], 'Te': next_result.var['Te'][-1], 'Ta': x_seq_copy['Ta'].iloc[t+2], 'Th': x_seq_copy['Th'].iloc[t+2], 'SIno': x_seq_copy['SIno'].iloc[t+2], 'SIso': x_seq_copy['SIso'].iloc[t+2] }

                modell_results_dict[pop_idx].append(new_row)
            
            # Evaluate Step
            if t > (pre_sequence_length+ignor_first_half_day):
                price_now = x_seq_copy.at[timestamp_t_plus_15, 'price']
                energy_taken = evaluate_energy(bkt_an,bkt_vorlauftemp_30,next_result.var['Tw'][-1],bkt_state_last, price_now) 
                e2_distance = evaluate_vorlauf_range(bkt_vorlauftemp) 
                e2_to_goal_temp = evaluate_temp(new_Ti,hour_with_minutes)
                reward_step = e2_to_goal_temp + e2_distance + energy_taken

                energy_X_days.append(energy_taken)
                distance_X_days.append(e2_distance)
                abweichung_temp_X_days.append(e2_to_goal_temp)
                reward_X_days.append(reward_step)

        # Evaluate example
        avg_energy_X_days = np.mean(energy_X_days)
        avg_distance_X_days = np.mean(distance_X_days)
        avg_abweichung_temp_X_days = np.mean(abweichung_temp_X_days)
        avgs_energy_X_days.append(avg_energy_X_days)
        avgs_distance_X_days.append(avg_distance_X_days)
        avgs_abweichung_temp_X_days.append(avg_abweichung_temp_X_days)
        
        reward_avg = np.mean(reward_X_days)
        rewards.append(reward_avg)
        reward_avgs.append(reward_avg)
    
    # Update reward history after each iteration
    for pop_idx in range(optimizer.pop_size):
        reward_history[pop_idx].append(reward_avgs[pop_idx])
    model_reward_history.append(avg_reward_seq_model)
    
    if (last_Iteration == True or i % update_feat_every == 0) and i > no_feat_imp_bevore:
        feat_res_copy = feat_results
        save_feat_csv(feat_res_copy)
        if last_Iteration == True:
            print("")
            print("##########  Done with RL  ###########")
            print("#####################################")
    
    if i % make_plot_each == 0 or take_test_data == True:
        
        print("  ")
        print(" Erstelle 3x2-Subplot-Raster ")

        # Create a 3x2 subplot grid
        fig, axs = plt.subplots(3, 2, figsize=(20, 15))
        axs = axs.flatten()  # Flache Liste für einfachen Zugriff
        
        # Adjust spacing between plots
        plt.subplots_adjust(hspace=0.35, wspace=0.17)

        # Fill the first 2x2 grid
        for pop_idx in range(optimizer.pop_size):
            modell_results = pd.DataFrame(modell_results_dict[pop_idx])
            if not modell_results.empty:
                for column in modell_results.columns[1:]:
                    color = color_scheme.get(column, 'black')  # Default to black if not found
                    """
                    if column in ['price_min_in_x','price_max_in_x']:
                        new_label = 'p_min' if column == 'price_min_in_x' else 'p_max'
                        axs[pop_idx].plot(modell_results['Timestamp'], modell_results[column] / 10, label=new_label, color=color, alpha=0.4)
                    elif column in ['price']:
                        axs[pop_idx].plot(modell_results['Timestamp'], modell_results[column] / 10, label=column, color=color)
                    elif column in ['SIno', 'SIso']:
                        axs[pop_idx].plot(modell_results['Timestamp'], modell_results[column] / 1000, label=column, color=color)
                    else:
                        axs[pop_idx].plot(modell_results['Timestamp'], modell_results[column], label=column, color=color)
                    """
                    if column in ['Ti','Th','Ta']:
                        new_label = 'default' 
                        if column == 'Ti':
                            new_label = 'Temp. Raum'
                        elif column == 'Th':
                            new_label = 'Temp. Heiz'
                        elif column == 'Ta':
                            new_label = 'Temp. Außen'

                        axs[pop_idx].plot(modell_results['Timestamp'], modell_results[column], label=new_label, color=color)
                        
                axs[pop_idx].fill_between(modell_results['Timestamp'], max_t_eval, min_t_eval, color='green', alpha=0.53)
                axs[pop_idx].legend(loc='upper right')
                axs[pop_idx].set_title(f'Population {pop_idx + 1} - Avg Reward: {reward_avgs[pop_idx]:.2f}\nTemp: {avgs_abweichung_temp_X_days[pop_idx]:.2f} - Energy: {avgs_energy_X_days[pop_idx]:.2f} - Range: {avgs_distance_X_days[pop_idx]:.2f}')
                axs[pop_idx].grid(True)  # Add grid
                axs[pop_idx].set_ylabel('Values')  # Add y-axis label
                axs[pop_idx].xaxis.set_major_locator(MaxNLocator(3))  # Limit x-axis ticks
        
        
        # Adjusted index range to match the range used in the loop for population data
        adjusted_start_idx = start_idx_w_pre   # Start from the first data point  pre_sequence_length
        adjusted_end_idx = end_idx_w_post  # End at the last data point

        # Extract timestamps from y_TRAIN for the relevant indices
        timestamps = y_TRAIN.index[adjusted_start_idx:adjusted_end_idx]
        
        # Create the DataFrame for seq_modell_df with the adjusted range
        seq_modell_data = {
            **{key: value[adjusted_start_idx:adjusted_end_idx] for key, value in result_TRAIN.X.items()},
            **{f'var_{key}': value[adjusted_start_idx:adjusted_end_idx] for key, value in result_TRAIN.var.items()},
            'BKT_an': input_df['BKT_an'][adjusted_start_idx:adjusted_end_idx]
        }
        seq_modell_df = pd.DataFrame(seq_modell_data)  

        # Add the 'Timestamp' column to seq_modell_df
        seq_modell_df['Timestamp'] = timestamps
        
        # Rename columns as specified
        seq_modell_df.rename(columns={'var_Ti': 'Ti', 'var_Te': 'Te', 'var_Tw': 'Tw'}, inplace=True)
        
        # Anwenden der shift()-Funktion und Ersetzen des ersten NaN-Wertes durch 1
        seq_modell_df['BKT_an_shifted'] = seq_modell_df['BKT_an'].shift(1).fillna(1)
        
        # Entfernen der ersten 12 Stunden aus seq_modell_df
        seq_modell_df_shortened = seq_modell_df.iloc[ignor_first_half_day:].copy()
        
        # Berechnen des durchschnittlichen Rewards
        energy_seq_model = [evaluate_energy(an, vorlauftemp, tw, last_state, price) 
                            for an, vorlauftemp, tw, last_state, price in zip(seq_modell_df_shortened['BKT_an'], 
                                                                       seq_modell_df_shortened['Th'], 
                                                                       seq_modell_df_shortened['Tw'], 
                                                                       seq_modell_df_shortened['BKT_an_shifted'],
                                                                       seq_modell_df_shortened['price'])]
        
        # Berechnen von 'hour_with_minutes' aus der 'Timestamp'-Spalte
        seq_modell_df_shortened['hour_with_minutes'] = [timestamp.hour + timestamp.minute / 60.0 for timestamp in seq_modell_df_shortened['Timestamp']]

        # Anpassen des Aufrufs von evaluate_temp
        temp_seq_model = [evaluate_temp(ti, hour_with_minutes) for ti, hour_with_minutes in zip(seq_modell_df_shortened['Ti'], seq_modell_df_shortened['hour_with_minutes'])]

        avg_energy_seq_model = np.mean(energy_seq_model)
        avg_temp_seq_model = np.mean(temp_seq_model)
        rewards_seq_model = avg_temp_seq_model + avg_energy_seq_model
        avg_reward_seq_model = rewards_seq_model

        # Plot the new script's data in the third row
        for column in ['Ti', 'Th', 'Tw', 'Ta', 'Te', 'SIno', 'SIso']:  # Ensure the order
            if column in seq_modell_df.columns:
                plot_data = seq_modell_df[column] / 1000 if column in ['SIno', 'SIso'] else seq_modell_df[column]
                axs[4].plot(seq_modell_df['Timestamp'], plot_data, label=column, color=color_scheme[column])
        
        # Add a grid, y-axis label, and limit x-axis ticks for the bottom plot
        axs[4].fill_between(modell_results['Timestamp'], max_t_eval, min_t_eval, color='green', alpha=0.2)
        axs[4].grid(True)
        axs[4].set_ylabel('Values')
        axs[4].xaxis.set_major_locator(MaxNLocator(3))
        axs[4].set_xlabel('Timestamp')
        axs[4].set_title(f'Seq Model - Avg Reward: {rewards_seq_model:.2f}\nTemp: {avg_temp_seq_model:.2f} - Energy: {avg_energy_seq_model:.2f}')
        axs[4].legend(loc='upper right')

        # Remove the unused sixth subplot
        fig.delaxes(axs[5])
        
        # Calculate and display moving averages
        moving_avg_rewards = {pop: np.mean(reward_history[pop][-5:]) if len(reward_history[pop]) >= 5 else np.mean(reward_history[pop]) for pop in range(optimizer.pop_size)}
        moving_avg_model_reward = np.mean(model_reward_history[-5:]) if len(model_reward_history) >= 5 else np.mean(rewards_seq_model)
 
        # Position for the moving average text
        text_x, text_y_start = 1.2, 0.8
        text_font_size = 20  # Adjust this value as needed
        delta_y = 0.13

        # Display moving averages for each population
        for pop_idx in range(optimizer.pop_size):
            axs[4].text(text_x, text_y_start - pop_idx * delta_y, 
                        f"Moving Avg Reward Pop {pop_idx + 1}: {moving_avg_rewards[pop_idx]:.2f}", 
                        transform=axs[4].transAxes, fontsize=text_font_size)

        # Display moving average for the model
        axs[4].text(text_x, text_y_start - len(reward_history) * delta_y, 
                    f"Moving Avg Reward Model: {moving_avg_model_reward:.2f}", 
                    transform=axs[4].transAxes, fontsize=text_font_size)
        
        plt.savefig(save_path + f'modell_results_combined_{i}.png')
        plt.close(fig)
        
        modell_results_dict = {pop: [] for pop in range(optimizer.pop_size)}
        
    if take_test_data == False:
        
        optimizer.tell(population, np.array(rewards), None, plot_path=save_path)
    else:
        # Check Test Run
        test_score_now = max(rewards)
        if i > 1000:
            if best_test_score < test_score_now:
                best_test_score = test_score_now
                best_test = i
                
        # Hinzufügen der Werte zu test_scores und test_scores_index
        test_scores.append(test_score_now)
        test_scores_index.append(i)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    2.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   31.8s
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   31.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.6s finished


#####################################
###########  Starting RL  ###########

 ########### I: 0 ############
  
 Erstelle 3x2-Subplot-Raster 

 ########### I: 1 ############

 ########### I: 2 ############
  
 Erstelle 3x2-Subplot-Raster 

 ########### I: 3 ############

 ########### I: 4 ############
  
 Erstelle 3x2-Subplot-Raster 
